In [1]:
from tqdm import tqdm
import random
import json
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2, ToTensor
from torchvision.models import resnet18

In [2]:
transforms = v2.Compose([
            v2.RandomHorizontalFlip(),
            v2.RandomRotation(10),
            v2.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
            v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            v2.ToTensor()
])

/home/cs/grad/sokhanka/miniconda3/envs/data-science/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
train_dataset = datasets.CIFAR100(
    root="./nodes/data",
    download=False,
    train=True,
    transform=transforms
)

test_dataset = datasets.CIFAR100(
    root="./nodes/data",
    download=False,
    train=False, 
    transform=ToTensor()
)

In [4]:
batch_size = 32
epochs = 100
num_classes = 100
learning_rate = 1e-3
weight_decay = 1e-4

In [5]:
val_size = len(test_dataset) // 2
val_dataset = torch.utils.data.Subset(test_dataset, range(val_size))
test_dataset = torch.utils.data.Subset(test_dataset, range(val_size, len(test_dataset)))


train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=val_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.resnet = resnet18(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.maxpool = nn.Identity()
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet18Classifier(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=3)

/home/cs/grad/sokhanka/miniconda3/envs/data-science/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cs/grad/sokhanka/miniconda3/envs/data-science/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
def train():
    model.train()
    losses = []
    model.train()
    for b, (X, y) in tqdm(enumerate(train_dataloader)):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
    return sum(losses)/len(losses)

In [9]:
def validate():
    model.eval()
    with torch.no_grad():
        X, y = next(iter(val_dataloader))
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
    return loss.item()

In [10]:
data = []
for e in range(epochs):
    loss = train()
    val_loss = validate()
    scheduler.step(val_loss)
    data.append({
        "epoch" : e + 1,
        "train_loss" : loss,
        "val_loss" : val_loss
    })

1563it [00:55, 28.22it/s]
1563it [00:54, 28.71it/s]
1563it [00:54, 28.53it/s]
1563it [00:54, 28.47it/s]
1563it [00:54, 28.65it/s]
1563it [00:54, 28.62it/s]
1563it [00:56, 27.83it/s]
1563it [00:54, 28.49it/s]
1563it [00:54, 28.48it/s]
1563it [00:55, 28.40it/s]
1563it [00:54, 28.49it/s]
1563it [00:54, 28.62it/s]
1563it [00:55, 28.29it/s]
1563it [00:54, 28.93it/s]
1563it [00:54, 28.79it/s]
1563it [00:54, 28.69it/s]
1563it [00:53, 29.14it/s]
1563it [00:54, 28.75it/s]
1563it [00:55, 28.20it/s]
1563it [00:55, 28.23it/s]
1563it [00:55, 28.27it/s]
1563it [00:54, 28.76it/s]
1563it [00:54, 28.80it/s]
1563it [00:54, 28.68it/s]
1563it [00:56, 27.82it/s]
1563it [00:55, 28.29it/s]
1563it [00:55, 28.40it/s]
1563it [00:55, 28.27it/s]
1563it [00:54, 28.48it/s]
1563it [00:54, 28.49it/s]
1563it [00:55, 28.26it/s]
1563it [00:54, 28.55it/s]
1563it [00:55, 28.35it/s]
1563it [00:55, 28.38it/s]
1563it [00:54, 28.46it/s]
1563it [00:54, 28.76it/s]
1563it [00:54, 28.69it/s]
1563it [00:54, 28.51it/s]
1563it [00:5

In [12]:
with open("./results/cifar100.json", "w") as f:
    f.write(json.dumps(data, indent=2))